## BPR simple

In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [3]:
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp
from subprocess import call
from bpr import *
# from bpr_module import *
# from cfbase import *
# from als import *
import matplotlib.pyplot as plt

In [4]:
cf = CFBase(
    users = [1, 1, 1, 2, 2, 2, 3, 3, 4, 4],
    items = [1, 2, 3, 4, 5, 6, 1, 2, 4, 5]
)
cf.generate_train_test(test_ratio=0.0)

In [5]:
als = ALS(
    num_iters = 10,
    num_features = 3,
    num_users = cf.R.shape[0],
    num_items = cf.R.shape[1],
    reg_lambda=0.
)

In [ ]:
als.fit(cf.R)

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
ax.plot(als.train_mse_record)

In [ ]:
np.around(als.predict(),0)

In [ ]:
cf.R.toarray()

In [ ]:
als.U

In [ ]:
cf.R_train.todense()

In [ ]:
cf.R.todense()

In [ ]:
print(cf)

In [ ]:
# user 1 and 2 no shared interactions
# user 3 has some overlap with user 1
# user 4 has some overlap with user 2
# viewed and clicked = 1
# viewed and not clicked = -1
# add veiwed and click (positive), viewed and not click (negative),
# learning only uses one pair at a time
df = pd.DataFrame({
    'UserID':      [1, 1, 1, 2, 2, 2, 3, 3, 4, 4], 
    'ProductID':   [1, 2, 3, 4, 5, 6, 1, 2, 4, 5],
    'Interaction': [1,-1, 1, 1,-1, 1, 1,-1, 1,-1]
    #'Interaction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
})
df

In [ ]:
X, df_new = create_simple_matrix(df, 'UserID', 'ProductID', 'Interaction')
df_new

In [ ]:
X.todense()

In [ ]:
X_train, X_test = create_train_test(X, test_size = 0.01, seed = 1234)
#X_train.todense()

In [ ]:
indptr = X.indptr
indices = X.indices
indptr, indices

In [ ]:
n_users, n_items = X.shape
n_users, n_items

In [ ]:
bpr_params = {
    'reg': 0.1,
    'learning_rate': 0.1,
    'n_iters': 500,
    'n_factors': 5,
    'batch_size': 1
}

In [ ]:
batch_iters = n_users//bpr_params['batch_size']
batch_iters

In [ ]:
rstate = np.random.RandomState(1234)
user_factors = np.around(rstate.normal(size=(n_users, bpr_params['n_factors'])),2)
item_factors = np.around(rstate.normal(size=(n_items, bpr_params['n_factors'])),2)
user_factors, item_factors

In [ ]:
X_start = user_factors @ item_factors.T
X_start

In [ ]:
batch_size = bpr_params['batch_size']
sampled_pos_items = np.zeros(batch_size, dtype=int)
sampled_users = np.random.choice(n_users, size=batch_size, replace=False)
sampled_pos_items, sampled_users

In [ ]:
X.todense()

In [ ]:
for idx, user in enumerate(sampled_users):
    pos_items = indices[indptr[user]:indptr[user + 1]]
    print(user, pos_items)

In [ ]:
def sample(n_users, n_items, indices, indptr, batch_size):
    """sample batches of random triplets u, i, j"""
    sampled_pos_items = np.zeros(batch_size, dtype=int)
    sampled_neg_items = np.zeros(batch_size, dtype=int)
    sampled_users = np.random.choice(n_users, size=batch_size, replace=False)
    for idx, user in enumerate(sampled_users):
        pos_items = indices[indptr[user]:indptr[user + 1]]
        pos_item = np.random.choice(pos_items)
        neg_item = np.random.choice(n_items)
        while neg_item in pos_items:
            neg_item = np.random.choice(n_items)
        sampled_pos_items[idx] = pos_item
        sampled_neg_items[idx] = neg_item
    return sampled_users, sampled_pos_items, sampled_neg_items

In [ ]:
sample(4, 6, indices, indptr, batch_size)

In [ ]:
np.zeros(1, dtype=int)

In [ ]:
for i in range(bpr_params['n_iters']):
    print(f'\n::::Iter-{i}')
    for j in range(batch_iters):
        print('SG-Batch-{j}')
        sampled = sample(n_users=n_users, n_items=n_items, indices=indices, indptr=indptr,
                         batch_size=bpr_params['batch_size'])
        sampled_users, sampled_pos_items, sampled_neg_items = sampled
        sampled_tuple = zip(sampled_users,sampled_pos_items, sampled_neg_items)
        user_u = user_factors[sampled_users]
        item_i = item_factors[sampled_pos_items]
        item_j = item_factors[sampled_neg_items]
        #print('User_matrix:\n', user_u)
        r_ui = np.diag(user_u.dot(item_i.T))
        r_uj = np.diag(user_u.dot(item_j.T))
        r_uij = r_ui - r_uj
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        for ixy, ixx in enumerate([(ix+1, iy+1, iz+1) for ix, iy, iz in sampled_tuple]):
            print(ixx, np.around(r_ui[ixy],2), np.around(r_uj[ixy],2), np.around(sigmoid[ixy],2))
        sigmoid_tiled = np.tile(sigmoid, (bpr_params['n_factors'], 1)).T
        #print(r_ui, r_uj, r_uij, sigmoid)
        grad_u = sigmoid_tiled * (item_j - item_i) + bpr_params['reg'] * user_u
        #grad_u = sigmoid_tiled * (0. - item_i) + bpr_params['reg'] * user_u
        grad_i = sigmoid_tiled * -user_u + bpr_params['reg'] * item_i
        grad_j = sigmoid_tiled * user_u + bpr_params['reg'] * item_j
        user_factors[sampled_users] -= bpr_params['learning_rate'] * grad_u
        item_factors[sampled_pos_items] -= bpr_params['learning_rate'] * grad_i
        #item_factors[sampled_neg_items] -= bpr_params['learning_rate'] * grad_j
#         self._update(sampled_users, sampled_pos_items,
#                      sampled_neg_items)

In [ ]:
np.around(user_factors,2)

In [ ]:
np.around(item_factors,2)

In [ ]:
this_user = 1
clicked_by_this_user = set(X[this_user].indices)
scores = user_factors[this_user].dot(item_factors.T)
np.around(scores,2)

In [ ]:
X.todense()